# Syncrhonized exploration of visual attention and verbalizations with fixation sequence and think-aloud protocols
- synchronizing fixations and protocols
- plotting them on the same timeline for interactive exploration


## synchronize fixations and protocol segments using their timestamps
Using files: 
- data/grouped_screen_replaced_mapped_fixation.tsv 
- data/protocol_segments.tsv  

Synchronization logic: for each fixaiton, find the protocol being verbalized at that moment. If there is no verbalization, then fill nan. 
             
Result file: data/sync_fixation_protocol.tsv

In [10]:
import pandas as pd 
import numpy as np
ta = pd.read_csv('data/segment_protocol.tsv', delimiter='\t')
et = pd.read_csv('data/grouped_screen_replaced_mapped_fixation.tsv', delimiter='\t')

offset_t = 119694  # time offset between protocol start and eye-tracking
ta['start'] = ta['start_time'] * 1000 + offset_t
ta['end'] = ta['end_time'] * 1000 + offset_t

for i in et.index:
    # find the ta row
    try:
        ta_index = ta[ta['start'].ge(et.at[i, 'Recording timestamp'])].index[0]
        et.at[i, 'protocol'] = ta.at[ta_index, 'content']
        et.at[i, 'ta_start'] = ta.at[ta_index, 'start']
        et.at[i, 'ta_dur'] = ta.at[ta_index, 'end'] - ta.at[ta_index, 'start']
    except:
        et.at[i, 'protocol'] = np.nan

et.to_csv('data/sync_fixation_protocol.tsv', sep='\t', index=False)

## plot fixations and protocols on the same timeline

In [4]:
import plotly.graph_objects as go
import plotly.io as pio

In [11]:
df = pd.read_csv('data/sync_fixation_protocol.tsv', sep='\t')
df.head()

,Recording timestamp,Eye movement type index,Gaze event duration,Fixation point X,Fixation point Y,target,phone_x,phone_y,best_match,screentime,new_target,protocol,ta_start,ta_dur
0,119832.0,105,620.0,1007.0,509.0,surroundings,NaN,NaN,NaN,0,surroundings,so I'm confused right now.,120784.0,1200.0
1,120222.0,106,400.0,857.0,356.0,cell phone,0.974220,0.210277,map_ar_2.jpg,88,map_ar,so I'm confused right now.,120784.0,1200.0
2,120842.0,107,80.0,1678.0,11.0,surroundings,NaN,NaN,NaN,0,surroundings,What this means.,121984.0,1960.0
3,121262.0,108,440.0,811.0,514.0,cell phone,0.573007,0.161089,map_ar_2.jpg,1128,map_ar,What this means.,121984.0,1960.0
4,123471.0,109,3799.0,919.0,312.0,cell phone,0.888017,0.129890,map_ar_2.jpg,3337,map_ar,What I want to tell you this way has he won't...,126004.0,2770.0


In [6]:
color_dict = {'info': 'rgb(253, 180, 98)',
              'map_ar': 'rgb(144, 211, 199)',  # green-ish
              'old_map': "rgb(255, 237, 111)", # yellow-ish
              'take_note': 'rgb(252, 205, 229)',
              'view_note': 'rgb(190, 186, 218)',
              'surroundings': 'rgb(204, 235, 197)',
              'building': 'rgb(128, 177, 211)', 
              'others': 'rgb(203, 203, 203)',
              'protocol segment': 'rgb(180, 180, 180)'}

In [12]:
df['start'] = (df['Recording timestamp'] - df['Gaze event duration']/2) / 1000
df['start'] = df['start'] - df['start'].min()
df['Gaze event duration'] = df['Gaze event duration'] / 1000
df['ta_start'] = df['ta_start'] / 1000
df['ta_start'] = df['ta_start'] - df['ta_start'].min()
df['ta_dur'] = df['ta_dur'] / 1000

In [13]:
fig = go.Figure(
        layout = {
        'barmode': 'stack',
        'xaxis': {'automargin': True, 'title': {'text': 'time into section (sec)'}},
        'yaxis': {'automargin': True,
                  'categoryorder': 'array',
                  'categoryarray': ['protocol segment', 'others','surroundings', 'building', 'view_note', 'take_note', 'map_ar', 'old_map', 'info']},
        'plot_bgcolor': 'rgb(250, 250, 250)'}
)
for target, target_df in df.groupby('new_target'):
    fig.add_bar(x=target_df['Gaze event duration'],
                y=target_df['new_target'],
                base=target_df.start,
                orientation='h',
                showlegend=False,
                marker=dict(color=color_dict[target]),
                name=target)

ta_df = df.groupby('protocol').agg(
    {'ta_start': 'min', 'ta_dur': 'median'}).reset_index()
ta_df['placeholder'] = 'protocol segment'
fig.add_bar(x=ta_df['ta_dur'],
            y=ta_df['placeholder'],
            base=ta_df.ta_start,
            orientation='h',
            showlegend=False,
            text=ta_df.protocol,
            hoverinfo='text',
            marker=dict(color=color_dict['protocol segment']))
fig.update_layout(hoverlabel=dict(bgcolor='rgb(250, 250, 250)', font_size=16))
fig.show()
pio.write_html(fig, 'plots/fixation_protocol.html')